# Save and restore models

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks

### Load data

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0   # (60000, 28, 28) --> (1000, 784)
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define the model

In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.keras.activations.softmax)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

### Save checkpoints during training

In [4]:
model = create_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
checkpoint_path = "ckpt/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = callbacks.ModelCheckpoint(checkpoint_path,
                                        save_weights_only=True,
                                        verbose=0)

In [6]:
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 302us/sample - loss: 2.3916 - acc: 0.0800
Untrained model, accuracy:  8.00%


In [7]:
model.fit(train_images, train_labels,
          epochs = 10,
          validation_data = (test_images, test_labels),
          verbose = 0,
          callbacks = [cp_callback])  # pass callback to training

In [8]:
# Loads the weights
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 55us/sample - loss: 0.4138 - acc: 0.8660
Restored model, accuracy: 86.60%


### Checkpoint callback options

In [9]:
model = create_model()

checkpoint_path = "ckpt/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = callbacks.ModelCheckpoint(checkpoint_path,
                                        save_weights_only=True,
                                        verbose=0,
                                        period=5)   # Save weights, every 5-epochs.

model.save_weights(checkpoint_path.format(epoch=0))

In [10]:
model.fit(train_images, train_labels,
          epochs = 20,
          validation_data = (test_images, test_labels),
          verbose = 0,
          callbacks=[cp_callback])

In [11]:
latest_ckpt = tf.train.latest_checkpoint(checkpoint_dir)
print(latest_ckpt)

ckpt\cp-0020.ckpt


In [12]:
# Load the previously saved weights
model = create_model()
model.load_weights(latest_ckpt)

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model 2, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 68us/sample - loss: 0.4352 - acc: 0.8740
Restored model 2, accuracy: 87.40%


### Manually save weights

In [13]:
# Save the weights
model.save_weights('ckpt/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('ckpt/my_checkpoint')

In [14]:
# Save the entire model
model.save('ckpt/my_model.h5')

Recreate the exact same model, including weights and optimizer...!!!

In [15]:
# Restore the entire model
new_model = keras.models.load_model('ckpt/my_model.h5')
new_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Reloaded entire model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 69us/sample - loss: 0.4352 - acc: 0.8740
Reloaded entire model, accuracy: 87.40%
